# 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
MIN_COUNTS = 20  ## 최소 단어 빈도 수
MAX_COUNTS = 1800 ## 최대 단어 빈도 수

MIN_LENGTH = 15 ## 문서가 최소한 가져야 하는 단어 개수

HALF_WINDOW_SIZE = 5 # 2*HALF_WINDOW_SIZE < MIN_LENGTH

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models
import random
import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [ ]:
nlp = spacy.load('en')

In [ ]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [ ]:
docs = [(i, doc) for i, doc in enumerate(docs)]

# 샘플 1000개 추출

In [ ]:
docs= random.sample(docs,k=1000)

In [ ]:
np.save('docs.npy', docs)

In [ ]:
docs[]

list

In [ ]:
type(dataset)

sklearn.utils.Bunch

In [ ]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 1000/1000 [00:42<00:00, 23.57it/s]

number of removed short documents: 214
total number of tokens: 72916
number of tokens to be removed: 37125
number of additionally removed short documents: 227
total number of tokens: 33521

minimum word count number: 14
this number can be less than MIN_COUNTS because of document removal


In [ ]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [ ]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

559it [00:00, 12470.23it/s]


In [ ]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [ ]:
# number of windows (equals to the total number of tokens)
data.shape[0]

33521

In [ ]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

In [ ]:
%%time
vocab_size = len(decoder)
embedding_dim = 10

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

CPU times: user 40.5 s, sys: 124 ms, total: 40.7 s
Wall time: 21.7 s


In [ ]:
vocab_size

736

In [ ]:
word_vectors.shape

(736, 10)

In [ ]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
%%time
n_topics = 5
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

CPU times: user 975 ms, sys: 29.5 ms, total: 1 s
Wall time: 985 ms


In [ ]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : use file like include need know good people mail year
topic 1 : know people think use drive like year find time system
topic 2 : system like time problem use work think know good run
topic 3 : know time people use right think like way good need
topic 4 : like know use problem file system good think find thing


In [ ]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 559/559 [00:00<00:00, 1118.44it/s]


In [ ]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)

In [ ]:
len(decoder)

736